In [ ]:
import add_packages
import yaml
from crewai import Agent, Task, Crew
from pprint import pprint
from my_crewai import tools

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
  configs_crewai = yaml.safe_load(f)

In [ ]:
origin = "Atlanta, GA"
travel_dates = "November 2024"
destination = "Thailand"
interests = "Hiking, Yoga, Sightseeing"

# Test

In [ ]:
class CrewTraveling():
  def __init__(
    self,
    destination,
    interests,
    origin,
    travel_dates,
  ) -> None:
    pass
    
    configs_agents_traveling = configs_crewai["agents"]["traveling"]
    configs_tasks_traveling = configs_crewai["tasks"]["traveling"]
    
    self.destination = destination
    self.interests = interests
    self.origin = origin
    self.travel_dates = travel_dates
    
    #* AGENTS ==================================================================
    self.agent_manager = Agent(
      **configs_agents_traveling["manager"],
      verbose=True,
      allow_delegation=True,
    )
    
    self.agent_travel_consultant = Agent(
      **configs_agents_traveling["travel_consultant"],
      verbose=True,
      tools=[
        tools.ToolsBrowser.search_serper,
        tools.ToolsCalculator.calculate,
      ],
      max_iter=7,
    )
    
    self.agent_city_selection_expert = Agent(
      **configs_agents_traveling["city_selection_expert"],
      verbose=True,
      tools=[
        tools.ToolsBrowser.search_duckduckgo,
        tools.ToolsContent.read_content,
      ],
    )
    
    self.agent_quality_control_expert = Agent(
      **configs_agents_traveling["quality_control_expert"],
      verbose=True,
      tools=[] + tools.tools_human,
    )
    
    #* TASKS ===================================================================
    self.task_manage = Task(
      **configs_tasks_traveling["manage"],
      agent=self.agent_manager,
      async_execution=False,
    )
    
    self.task_identify_city = Task(
      **configs_tasks_traveling["identify_city"],
      agent=self.agent_city_selection_expert,
      async_execution=False,
    )
    
    self.task_gather_city_info = Task(
      **configs_tasks_traveling["gather_city_info"],
      agent=self.agent_city_selection_expert,
      async_execution=False,
    )
    
    self.task_plan_itinerary = Task(
      **configs_tasks_traveling["plan_itinerary"],
      agent=self.agent_travel_consultant,
      async_execution=False,
    )
    
    self.task_quality_control = Task(
      **configs_tasks_traveling["quality_control"],
      agent=self.agent_quality_control_expert,
      context=[
        self.task_plan_itinerary
      ],
      async_execution=False,
    )
    #*==========================================================================
    
    self.crew = Crew(
      agents=[
        self.agent_manager,
        self.agent_travel_consultant,
        self.agent_city_selection_expert,
        self.agent_travel_consultant,
        self.agent_quality_control_expert,
      ]
    )
    #*==========================================================================
    
    self._process()
    
  def _process(self):
    self.agent_manager.goal = self.agent_manager.goal.replace(
      "%DESTINATION%", self.destination
    )
    
    self.agent_travel_consultant.goal = self.agent_travel_consultant.goal.replace(
      "%DESTINATION%", self.destination
    )
    
    self.task_manage.description = self.task_manage.description\
      .replace("%INTERESTS%", self.interests)\
      .replace("%DESTINATION%", self.destination)
    
    self.task_identify_city.description = self.task_identify_city.description\
      .replace("%ORIGIN%", self.origin)\
      .replace("%DESTINATION%", self.destination)\
      .replace("%TRAVEL_DATES%", self.travel_dates)\
      .replace("%INTERESTS%", self.interests)
      
    self.task_gather_city_info.description = self.task_gather_city_info.description\
      .replace("%DESTINATION%", self.destination)\
      .replace("%TRAVEL_DATES%", self.travel_dates)\
      .replace("%INTERESTS%", self.interests)
    
    self.task_plan_itinerary.description = self.task_plan_itinerary.description\
      .replace("%ORIGIN%", self.origin)\
      .replace("%DESTINATION%", self.destination)\
      .replace("%TRAVEL_DATES%", self.travel_dates)\
      .replace("%INTERESTS%", self.interests)
    self.task_plan_itinerary.expected_output = self.task_plan_itinerary.expected_output\
      .replace("%DESTINATION%", self.destination)\

crew_traveling = CrewTraveling(
  destination=destination,
  interests=interests,
  origin=origin,
  travel_dates=travel_dates,
)

# Test 1

In [63]:
import yaml
import re


class CustomCrew():
  def __init__(self, configs, task_name, **kwargs) -> None:
    # Extract parameters from kwargs
    for key, value in kwargs.items():
      setattr(self, key, value)

    # Load configurations from crewai.yaml
    configs_agents: dict = configs["agents"][task_name]
    configs_tasks: dict = configs["tasks"][task_name]

    # Initialize agents
    self.agents = {}
    for agent_key, agent_config in configs_agents.items():
      # Perform placeholder substitution
      agent_config = self.substitute_placeholders(agent_config)
      # Initialize each agent using Agent class
      self.agents[agent_key] = Agent(**agent_config)

    # Initialize tasks
    self.tasks = {}
    for task_key, task_config in configs_tasks.items():
      # Perform placeholder substitution
      task_config = self.substitute_placeholders(task_config)
      self.tasks[task_key] = {key: task_config[key]
                              for key in task_config.keys()}

  def substitute_placeholders(self, config):
    # Regex pattern to match placeholders like %DESTINATION%
    pattern = r'%(\w+)%'
    # Function to replace placeholders with actual values

    def repl(match):
      placeholder = match.group(1)
      # Check if the placeholder matches any attribute of the class
      if hasattr(self, placeholder.lower()):
        return str(getattr(self, placeholder.lower()))
      return match.group(0)  # Return the placeholder if not found

    # Perform substitution on each value in the config dictionary
    return {key: re.sub(pattern, repl, value) for key, value in config.items()}

  def __str__(self):
    output = "Crew Information:\n\n"

    output += "Agents:\n"
    for agent_key, agent in self.agents.items():
      output += f"- {agent_key}:\n"
      output += f"-- Role: {agent.role}\n"
      output += f"-- Goal: {agent.goal}"
      output += f"-- Backstory: {agent.backstory}\n"

    output += "Tasks:\n"
    for task_key, task in self.tasks.items():
      output += f"- {task_key}:\n"
      for key, value in task.items():
        output += f'-- {key}:\n"\n{value}"\n'
      output += "\n"
    return output


# Example usage
destination = "Paris"
origin = "New York"
interests = ["Art", "Cuisine"]
travel_dates = "2024-06-01 to 2024-06-10"

# Load configurations from YAML file
with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as file:
  configs_crewai = yaml.safe_load(file)

# Initialize CrewTraveling instance
crew_traveling = CustomCrew(
  configs_crewai, 
  task_name="traveling", 
  destination=destination,
  origin=origin, 
  interests=interests, 
  travel_dates=travel_dates
)

# Print crew information
print(crew_traveling)

Crew Information:

Agents:
- manager:
-- Role: Travel Manager
-- Goal: Coordinate the trip to Paris ensure a seamless integration of research findings into a comprehensive travel report with daily activities, budget breakdown, and packing suggestions.
-- Backstory: With a strategic mindset and a knack for leadership, you excel at guiding teams towards their goals, ensuring the trip not only meets but exceed expectations. You also validate your final output before presenting it to the client.

- travel_consultant:
-- Role: Amazing Travel Consultant
-- Goal: Create the most amazing travel itineraries with budget and packing suggestions for your clients to travel to Paris.
-- Backstory: Specialist in travel planning and logistics with decades of experience

- city_selection_expert:
-- Role: City Selection Expert
-- Goal: Select the best city based on weather, season, and prices
-- Backstory: An expert in analyzing travel data to pick ideal destinations

- local_tour_guide:
-- Role: Local 